<img src="assets/Orchestrator.png" align="left" width="600" style="margin-right:15px;"/>

Orchestrator-worker workflows provide more flexibility and are often used when subtasks cannot be predefined the way they can with parallelization. This happens with we cannot predetermine the tasks.

In an orchestrator-worker configuration, the orchestrator:
Breaks down tasks into subtasks
Delegates subtasks to workers
Synthesizes worker outputs into a final result

Orchestrator-worker workflows are common and LangGraph has built-in support for them. The Send API lets you dynamically create worker nodes and send them specific inputs. Each worker has its own state, and all worker outputs are written to a shared state key that is accessible to the orchestrator graph. This gives the orchestrator access to all worker output and allows it to synthesize them into a final output. The example below iterates over a list of sections and uses the Send API to send a section to each worker.

In [ ]:
from dotenv import load_dotenv, find_dotenv
from env_utils import doublecheck_env
from langchain_anthropic import ChatAnthropic
from langchain.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END

path = find_dotenv()
print("Loaded env from:", path)

# Load environment variables from .env
load_dotenv(find_dotenv())

# Check and print results
doublecheck_env(path)

llm = ChatAnthropic(model="claude-sonnet-4-5-20250929")

In [ ]:
from typing import Annotated, List
import operator
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-sonnet-4-5-20250929")

# Schema for structured output to use in planning
class Section(BaseModel):
    name: str = Field(
        description="Name for this section of the report.",
    )
    description: str = Field(
        description="Brief overview of the main topics and concepts to be covered in this section.",
    )


class Sections(BaseModel):
    sections: List[Section] = Field(
        description="Sections of the report.",
    )


# Augment the LLM with schema for structured output
planner = llm.with_structured_output(Sections)

In [ ]:
out = planner.invoke('write a report for langgarph')

In [ ]:
print(out)

In [ ]:
from langgraph.types import Send
from typing_extensions import TypedDict
from IPython.display import display, Image


# Graph state
class State(TypedDict):
    topic: str  # Report topic
    sections: list[Section]  # List of report sections
    completed_sections: Annotated[
        list, operator.add
    ]  # All workers write to this key in parallel
    final_report: str  # Final report


# Worker state
class WorkerState(TypedDict):
    section: Section
    completed_sections: Annotated[list, operator.add]


# Nodes
def orchestrator(state: State):
    """Orchestrator that generates a plan for the report"""

    # Generate queries
    report_sections = planner.invoke(
        [
            SystemMessage(content="Generate a plan for the report."),
            HumanMessage(content=f"Here is the report topic: {state['topic']}"),
        ]
    )

    return {"sections": report_sections.sections}


def llm_call(state: WorkerState):
    """Worker writes a section of the report"""

    # Generate section
    section = llm.invoke(
        [
            SystemMessage(
                content="Write a concise report section following the provided name and description. IMPORTANT: Keep the section to a maximum of 3 sentences. Include no preamble. Use markdown formatting."
            ),
            HumanMessage(
                content=f"Here is the section name: {state['section'].name} and description: {state['section'].description}"
            ),
        ]
    )

    # Write the updated section to completed sections
    return {"completed_sections": [section.content]}


def synthesizer(state: State):
    """Synthesize full report from sections"""

    # List of completed sections
    completed_sections = state["completed_sections"]

    # Format completed section to str to use as context for final sections
    completed_report_sections = "\n\n---\n\n".join(completed_sections)

    return {"final_report": completed_report_sections}


# Conditional edge function to create llm_call workers that each write a section of the report
def assign_workers(state: State):
    """Assign a worker to each section in the plan"""

    # Kick off section writing in parallel via Send() API
    return [Send("llm_call", {"section": s}) for s in state["sections"]]


# Build workflow
orchestrator_worker_builder = StateGraph(State)

# Add the nodes
orchestrator_worker_builder.add_node("orchestrator", orchestrator)
orchestrator_worker_builder.add_node("llm_call", llm_call)
orchestrator_worker_builder.add_node("synthesizer", synthesizer)

# Add edges to connect nodes
orchestrator_worker_builder.add_edge(START, "orchestrator")
orchestrator_worker_builder.add_conditional_edges(
    "orchestrator", assign_workers, ["llm_call"]
)
orchestrator_worker_builder.add_edge("llm_call", "synthesizer")
orchestrator_worker_builder.add_edge("synthesizer", END)

# Compile the workflow
orchestrator_worker = orchestrator_worker_builder.compile()

# Show the workflow
display(Image(orchestrator_worker.get_graph().draw_mermaid_png()))

# Invoke
# state = orchestrator_worker.invoke({"topic": "Create a report on LLM scaling laws"})

# from IPython.display import Markdown
# Markdown(state["final_report"])

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# Clear cell output
clear_output(wait=True)

# Create all widgets fresh
topic_input = widgets.Text(
    value='Create a report on LLM scaling laws',
    placeholder='Enter your report topic...',
    description='Topic:',
    continuous_update=False,
    layout=widgets.Layout(width='500px')
)

submit_btn = widgets.Button(
    description='Generate Report',
    button_style='primary',
    icon='file-text'
)

result_html = widgets.HTML(value='')

def on_button_click(btn):
    """Handle button click"""
    topic = topic_input.value.strip()

    if not topic:
        result_html.value = "<p>⚠️ Please enter a report topic!</p>"
        return

    # Show loading state
    btn.disabled = True
    result_html.value = f"<p>🔄 Generating report on: {topic}...</p>"

    try:
        # Invoke workflow
        state = orchestrator_worker.invoke({"topic": topic})

        # Get the sections and final report
        sections = state.get('sections', [])
        final_report = state.get('final_report', '')

        # Build section list HTML
        section_list = ""
        for i, section in enumerate(sections, 1):
            section_list += f"<li><strong>{section.name}</strong>: {section.description}</li>"

        # Build HTML string with results
        html = f"""
        <div style="font-family: 'Segoe UI', Arial, sans-serif; padding: 15px; background: #f8f9fa; border-radius: 5px;">
            <h3 style="color: #0066cc; margin-top: 0;">📄 Report: {topic}</h3>
            <hr style="border: 1px solid #0066cc;">

            <div style="margin: 20px 0;">
                <h4 style="color: #0066cc;">Generated Sections ({len(sections)})</h4>
                <div style="background: white; padding: 15px; border-left: 3px solid #0066cc; margin-top: 10px;">
                    <ul>{section_list}</ul>
                </div>
            </div>

            <div style="margin: 20px 0;">
                <h4 style="color: #0066cc;">Final Report</h4>
                <div style="background: white; padding: 15px; border-left: 3px solid #28a745; margin-top: 10px;">
                    <div style="white-space: pre-wrap; font-family: 'Segoe UI', Arial, sans-serif;">{final_report}</div>
                </div>
            </div>
        </div>
        """
        result_html.value = html

    except Exception as e:
        result_html.value = f"<p>❌ Error: {str(e)}</p>"

    finally:
        btn.disabled = False

# Attach handler
submit_btn.on_click(on_button_click)

# Display
display(widgets.VBox([
    widgets.HTML("<h3>📝 Orchestrator-Worker Report Generator</h3>"),
    widgets.HTML("""<p>Enter a topic and the system will orchestrate multiple workers to generate a comprehensive report:</p>
    <ul>
        <li><strong>Orchestrator:</strong> Creates a plan and breaks down the report into sections</li>
        <li><strong>Workers:</strong> Each worker writes one section of the report in parallel</li>
        <li><strong>Synthesizer:</strong> Combines all sections into a final comprehensive report</li>
    </ul>
    <p><em>Examples: "LLM scaling laws", "Impact of climate change on agriculture", "Quantum computing applications"</em></p>"""),
    widgets.HBox([topic_input, submit_btn]),
    result_html
]))